# Maxar Stereo Data 

This notebooks illustrates basic search and visualization of Maxar Stereo Imagery.

Coincident uses the [Maxar 'discovery' STAC API](https://developers.maxar.com/docs/discovery/) to find high resolution stereo data.

In [ ]:
import coincident
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline

## Basic search

Below we use a simple bounding box, date range, and filter to find scenes with cloud cover less than 25%

In [ ]:
bbox = [-71.8, 42.0, -71.5, 42.1]

gf_stereo = coincident.search.search(
    dataset="maxar",
    bbox=bbox,
    datetime=["2021-03-08", "2021-04-01"],
    filter="eo:cloud_cover < 25",
)
len(gf_stereo)

In [ ]:
# Print all the metadata for a particular scene
# Note 'stereo_pair_identifiers' and 'id' for ordering full-resolution data
with gpd.pd.option_context(
    "display.max_rows", None, "display.max_columns", None
):  # more options can be specified also
    print(gf_stereo.iloc[0])

In [ ]:
popcols = ["title", "eo:cloud_cover", "datetime", "view:azimuth", "view:off_nadir"]
gf_stereo.explore(popup=popcols)

## Load browse image 

Maxar provides monochrome uint8 or RGB browse images at ~30m resolution that we can visualize before ordering the full-resolution scenes. 

In [ ]:
# We first convert from geopandas metadata to a pystac items
items = coincident.search.to_pystac_items(gf_stereo)

In [ ]:
da = coincident.io.xarray.open_maxar_browse(items[0]).squeeze()
da

## Plot browse image

For convenience you can create simplified figures directly without first loading the data with Xarray. Browse images are cloud-optimized-geotiffs with overviews for efficient previewing.

In [ ]:
coincident.plot.plot_maxar_browse(items[0], overview_level=2);

## Advanced Search

Optical imagery often has estimates of cloud-cover in the metadata. This is typically a single percentage value for the entire scene footprint. The [Maxar API](https://developers.maxar.com/docs/discovery/guides/discovery-guide#area-based-calculations) goes a step further allowing for filtering cloudcover for *only* the AOI rather than the entire scene footprint.

![](https://developers.maxar.com/assets/images/area-based-calc-56bf8d3e6cddc55ae8dcc36ddc4f41c6.png)


Activating the "area_based_calc" search feature through coincident requires modifying the default "maxar" dataset class. Once activated, in addition to 'eo:cloud_cover' 'view:off_nadir' metadata will include 'area:cloud_cover_percentage' and 'area:avg_off_nadir_angle'

Note that these searches will take longer!

In [ ]:
maxar = coincident.datasets.maxar.Stereo()
maxar

In [ ]:
# Use AOI-based estimates of cloud-cover instead of entire scene
maxar.area_based_calc = True

In [ ]:
gf = coincident.search.search(
    dataset=maxar,
    bbox=bbox,
    datetime=["2021-03-08", "2021-04-01"],
)

cols = [
    "title",
    "eo:cloud_cover",
    "area:cloud_cover_percentage",
    "view:off_nadir",
    "area:avg_off_nadir_angle",
]
gf[cols]

In [ ]:
# Plot with cloud-cover
items = coincident.search.to_pystac_items(gf)
item = items[0]

ax = coincident.plot.plot_maxar_browse(item, overview_level=1)
# Add AOI
rect = plt.Rectangle(
    (bbox[0], bbox[1]),
    bbox[2] - bbox[0],
    bbox[3] - bbox[1],
    linewidth=2,
    edgecolor="magenta",
    facecolor="none",
)
ax.add_patch(rect)
plt.autoscale()

## Load cloud-cover polygon

For each acquisition, Maxar includes a Multipolygon that estimates cloud cover. Behind the scenes, coincident uses the stac-asset library to read and download assets of any STAC item. Because stac-asset uses asynchronous functions, you have to use `await` to return results.

In [ ]:
bytes = await coincident.io.download.read_href(item, "cloud-cover")
gf_clouds = gpd.read_file(bytes)
gf_clouds.explore()

## Download browse image + metadata

coincident wraps the stac-asset library to easily download local copies of a STAC item and all of its assets (e.g. metadata and cloud cover mask)

In [ ]:
# Set download directory to match item STAC ID
download_dir = f"/tmp/{item.id}"
local_item = await coincident.io.download.download_item(item, download_dir)

In [ ]:
!ls {download_dir}